In [8]:
import sys
sys.path.append("../")
import pandas as pd
import re

from urllib.parse import urlparse
import pandas as pd
from mastodon import Mastodon
from datetime import datetime
from desci_sense.shared_functions.schema.post import RefPost
from desci_sense.shared_functions.dataloaders.mastodon.mastodon_loader import MastodonLoader
from desci_sense.shared_functions.dataloaders.mastodon.filter_timeline import FilterTimeline
from desci_sense.shared_functions.dataloaders.mastodon.check_publication import CheckPublication 
from desci_sense.shared_functions.web_extractors.citoid import fetch_citation

In [3]:
# create mastodon api token (https://sohwatt.com/generating-a-token-on-mastodon/)
# and set as environment variable using `export MASTODON_ACCESS_TOKEN="your_access_token_here"`
mloader = MastodonLoader()
ftimeline=FilterTimeline()

In [4]:
accts = ["@ronent@mastodon.social"]
posts = mloader.load_profiles(mastodon_accounts=accts)

In [9]:
start_date = datetime(2024, 1, 2)  
end_date = datetime(2024, 2, 2) 
posts_1 = ftimeline.filter_timeline(mastodon_accounts=accts, start_date=start_date, end_date=end_date)
posts_1

[{'id': 111822571204212368,
  'content': '<p>Also relevant for discussions about science social media as critical scientific infrastructure <span class="h-card" translate="no"><a href="https://fediscience.org/@UlrikeHahn" class="u-url mention">@<span>UlrikeHahn</span></a></span>  <span class="h-card" translate="no"><a href="https://mastodon.social/@brembs" class="u-url mention">@<span>brembs</span></a></span> <span class="h-card" translate="no"><a href="https://social.coop/@jonny" class="u-url mention">@<span>jonny</span></a></span>  <span class="h-card" translate="no"><a href="https://indieweb.social/@bonfire" class="u-url mention">@<span>bonfire</span></a></span> indieweb.social  <span class="h-card" translate="no"><a href="https://mas.to/@nanopub" class="u-url mention">@<span>nanopub</span></a></span>  </p><p>&quot; TikTokification of academia&quot;</p><p><a href="https://arxiv.org/abs/2401.13782" target="_blank" rel="nofollow noopener noreferrer" translate="no"><span class="invisib

In [10]:
df = pd.DataFrame(posts_1)


# Define a regular expression pattern for URLs
url_pattern = r'https?://[^\s<>"]+|www\.[^\s<>"]+'

# Function to extract URLs from a given text
def extract_urls(text):
    url_pattern = r'https?://[^\s<>"]+|www\.[^\s<>"]+'
    return re.findall(url_pattern, text)

# Apply the function to the 'content' column of your DataFrame
df['urls'] = df['content'].apply(extract_urls)

# Now 'urls' column will contain lists of extracted URLs
df[['content', 'urls']]

,content,urls
0,<p>Also relevant for discussions about science...,"[https://fediscience.org/@UlrikeHahn, https://..."
1,"<p>Thanks for reading! Happy new year, make me...",[]
2,<p>Shout-outs to some awesome ppl/orgs we&#39;...,[https://twitter.com/rtk254/status/17418417478...
3,<p>Our goal is not to build yet another social...,[]
4,<p>Different from services that track research...,[https://twitter.com/rtk254/status/15287420514...


In [11]:
import sys
sys.path.append("../")
from desci_sense.shared_functions.web_extractors.citoid import fetch_citation

target_url = " https://arxiv.org/abs/2401.13782"

fetch_citation(target_url)

{'key': 'MGHP6NE2',
 'version': 0,
 'itemType': 'preprint',
 'creators': [{'firstName': 'Iain Xie',
   'lastName': 'Weissburg',
   'creatorType': 'author'},
  {'firstName': 'Mehir', 'lastName': 'Arora', 'creatorType': 'author'},
  {'firstName': 'Liangming', 'lastName': 'Pan', 'creatorType': 'author'},
  {'firstName': 'William Yang', 'lastName': 'Wang', 'creatorType': 'author'}],
 'tags': [{'tag': 'Computer Science - Digital Libraries', 'type': 1},
  {'tag': 'Computer Science - Artificial Intelligence', 'type': 1},
  {'tag': 'Computer Science - Computation and Language', 'type': 1},
  {'tag': 'Computer Science - Computer Vision and Pattern Recognition',
   'type': 1},
  {'tag': 'Computer Science - Machine Learning', 'type': 1},
  {'tag': 'Computer Science - Social and Information Networks', 'type': 1}],
 'title': 'Tweets to Citations: Unveiling the Impact of Social Media Influencers on AI Research Visibility',
 'date': '2024-01-24',
 'abstractNote': "As the number of accepted papers at 

In [12]:
def check_publication(row):
    # Assume the row['urls'] is a list of URLs
    for url in row['urls']:
        if fetch_citation(url):
            return 1
    return 0
df['publication'] = df.apply(check_publication, axis=1)
df

,id,content,created_at,account_id,username,urls,publication
0,111822571204212368,<p>Also relevant for discussions about science...,2024-01-26 13:50:26.443000+00:00,107714271376772838,ronent,"[https://fediscience.org/@UlrikeHahn, https://...",1
1,111687085875576875,"<p>Thanks for reading! Happy new year, make me...",2024-01-02 15:34:44.379000+00:00,107714271376772838,ronent,[],0
2,111687084670741683,<p>Shout-outs to some awesome ppl/orgs we&#39;...,2024-01-02 15:34:25.928000+00:00,107714271376772838,ronent,[https://twitter.com/rtk254/status/17418417478...,1
3,111687079492648450,<p>Our goal is not to build yet another social...,2024-01-02 15:33:06.983000+00:00,107714271376772838,ronent,[],0
4,111687078527305926,<p>Different from services that track research...,2024-01-02 15:32:52.254000+00:00,107714271376772838,ronent,[https://twitter.com/rtk254/status/15287420514...,1


In [13]:
import pandas as pd
from urllib.parse import quote
import requests

# Assuming the fetch_citation function is defined as before and it returns None if the citation is not found

def check_publication_and_store_metadata(row):
    metadata_columns = {}
    # Iterate over each URL in the 'urls' list within the row
    for i, url in enumerate(row['urls']):
        citation = fetch_citation(url)
        # Check if the citation is valid (not an error message)
        if citation and 'DOI' in citation:
            # Store the metadata in a dictionary with a dynamic key based on the URL index
            metadata_columns[f'url_{i+1}_meta_info'] = citation
        else:
            # If no valid citation was found, use None or an empty dict, based on your preference
            metadata_columns[f'url_{i+1}_meta_info'] = None
    
    # If no valid citations were found at all, set 'publication' to 0
    if not any(metadata_columns.values()):
        row['publication'] = 0
    else:
        # If at least one valid citation was found, set 'publication' to 1
        row['publication'] = 1
        # Update the row with the metadata columns
        row.update(metadata_columns)
    
    return row



# Apply the function to each row
df = df.apply(check_publication_and_store_metadata, axis=1)

# Display the DataFrame
print(df)

                   id                                            content  \
0  111822571204212368  <p>Also relevant for discussions about science...   
1  111687085875576875  <p>Thanks for reading! Happy new year, make me...   
2  111687084670741683  <p>Shout-outs to some awesome ppl/orgs we&#39;...   
3  111687079492648450  <p>Our goal is not to build yet another social...   
4  111687078527305926  <p>Different from services that track research...   

                        created_at          account_id username  \
0 2024-01-26 13:50:26.443000+00:00  107714271376772838   ronent   
1 2024-01-02 15:34:44.379000+00:00  107714271376772838   ronent   
2 2024-01-02 15:34:25.928000+00:00  107714271376772838   ronent   
3 2024-01-02 15:33:06.983000+00:00  107714271376772838   ronent   
4 2024-01-02 15:32:52.254000+00:00  107714271376772838   ronent   

                                                urls  publication  
0  [https://fediscience.org/@UlrikeHahn, https://...            1  
1   

In [14]:
import pandas as pd
import requests
from urllib.parse import quote

def store_citation_metadata(urls):
    # Initialize an empty list to hold the metadata
    metadata_list = []

    # Loop over each URL and fetch its citation metadata
    for url in urls:
        metadata = fetch_citation(url)
        if metadata and 'DOI' in metadata:  # Check for valid metadata with a 'DOI'
            metadata_list.append(metadata)

    return metadata_list
### define the useful metadata infomation 